### Получение сборок.
1. Получение двух фаста файлов, списка всех UCE, файл со всеми ридами. 
   
   (cat IonXpress_\d\d\d_rawlib.basecaller_unadapt_trimmed0_0.cor.fasta > All_spicies.fasta)

    -> All_spicies.fasta
    -> All_UCE.fasta
    
2. Сборка de novo через Trinity. 

   (Trinity --seqType fa --single All_spicies.fasta) 
   
   -> All_spicies_Trinity.fasta
    
3. Поиск включений UCE в сборке через бласт (через биопитон?). 
   
   (makeblastdb  -in 'All_spicies_Trinity.fasta' -dbtype nucl -out blast_index, 
   
   blastn -query All_UCE.fasta -db blast_index -outfmt 6 -out UCE_in_Trinity.outfmt6)
   
   -> UCE_in_Trinity.outfmt6
   
4. Из UCE_in_Trinity.outfmt6 и All_spicies_Trinity.fasta составление фаста файлов с контигами, содержащими UCE, для каждого UCE.

   (with open UCE_in_Trinity.outfmt6:
     по имени UCE в первой строчке, создаем фаста файл, куда записываем из файла All_spicies_Trinity.fasta контиги, найденные в UCE_in_Trinity.outfmt6 по шаблону второй строки)
   
   -> UCE№_by_Trinity.fasta (по числу UCE)
   
5. Полученные файлы помещаю в CD-HIT-EST 
   
   (cd-hit-est -i UCE№_by_Trinity.fasta -c 0.8 -l 200 -r 1 -d 30)
   
6. Делаю фаста файлы с кластерами. 
   
   (в файле )
   
   -> UCE№_by_Trinity_Claster№.fasta (по числу кластеров для каждого UCE * число UCE)
   
7. Загружаю файлы с кластерами в сар3. 
   
   (cap3 UCE№_by_Trinity_Claster№.fasta  -p 85 -f 1500 -e 500 -z 1 -o 16 -s 251)
   
   -> UCE№_by_Trinity_cap3_Cosensus№.fasta
 
8. Для каждого консенсуса делаю сборку на него каждого из видов в BWA. 
   
   (bwa index UCE№_by_Trinity_cap3_Cosensus№.fasta
   
   bwa mem UCE№_by_Trinity_cap3_Cosensus№.fasta SP№.fasta > UCE№_SP№_Сonsensus№.sam
   samtoola view -F 4 UCE№_SP№_Сonsensus№.sam > UCE№_SP№_Сonsensus№_mapping.sam)
   
   -> UCE№_SP№_Сonsensus№_mapping.sam

Два основных модуля. 
1. Получение сборок
2. Чистка сборок

### Чистка сборок (прописано для одной сборки, повторяем для всех)
Наверно через биопитон или через pysam

1. Делаем список, со значениями покрытия для каждого нуклеотида. (samtools depth) (для частоты встерчаемости замен)

Готово!

2. Для каждого рида выписываем замены в словарь (Тип замены: Номер нуклеотида)

3. Сет из всех возможных замен. Считаем частоту этих замен, сравнивая с покрытием
   
4. Если замена харктерна для 10%-40% покрытия, возможно, риды с такое заменой принадлежат другой хромосоме. (их и ищем, чтобы удалить)
   
   Если замена характерна для 40%-60% покрытия, это скорее всего гетерозиготность. (или разница A и B хромосом)
   
   Если замена характерна для 100% покрытия, это отличие от референса, характерное для вида. (Этот тип замен можно удалить из словарей замен)
   
   Если замена характерна для <10% покрытия, это возможно ошибки в гомополимерах. (проверяем соседние нуклеотиды, если справа или слева идут такие же нуклеотиды, то скорее всего    это просто ошибка)
   
   Если замена характерна для 60% < 90% покрытия, возможно это тоже межвидовые различия
   
   

5. Смотрим на замены характерные менее чем для 10 % ридов. Если в словарях, где есть эти замены, есть еще замены менее 10%, то рид выкидываем.


Что делать если рида в сборке всего два?
Call_bases
Очистить sam файл, тогда колонок будет одинакого. Убрать первую строку.

In [1]:
import pysam
import Bio
import pandas as pd
from Bio import SeqIO

In [12]:
# Загружаем файл, берем из него колонки с ридом, и позицией начала рида Флаги!
def data():
    data = pd.read_csv('UCE4_sp3_trinity_cap3_contig7_cart.sam', sep = '\t')
    data = pd.concat([pd.DataFrame([data.columns.values], columns=data.columns), data], ignore_index=True) # сдвигаем данные (название столбцов это часть данных)


    start = data.iloc[:, 3] # столбец из sam, где начало рида
    seq = data.iloc[:, 9] # столбец из sam c самим ридом
    cigar = data.iloc[:, 5] # столбец из sam с строкой cigar (ищем гапы)
    flag = data.iloc[:, 1] # столбец из sam с строкой cigar (ищем гапы)

    data =  start.to_frame().join(seq.to_frame()).join(cigar.to_frame()).join(flag.to_frame())
    data.columns = ['Начало рида', 'Рид', 'CIGAR','Flag']
    data['Начало рида'].iloc[0] = int(data['Начало рида'].iloc[0])
    return data

In [13]:
# загрузим соответствующий референс
ref_fasta= SeqIO.parse(open('UCE4_contig7.fasta'),'fasta')
for i in ref_fasta:
    bio_ref = i.seq
ref =[]
for i in bio_ref:
    ref.append(i)
#ref

Цель: Получить словари, где число элементов словаря, число нуклеотидов в референсе. Каждый элемент тоже словарь, {Нуклеотид 1: {A: count A, C: count C, T: count T; G: count G}, ...}

При получении информации о нуклеотидах важно: Смотрим на позицию начала рида. Смотрим на cigar, гапы пропускаем, пропускаем и soft clipping.

In [14]:
ref_cover_dict = [] # большой словарь для всех нуклеотидов, по всей длине референса
for i in range(len(ref) + 1):
    cover_dict = {} # для каждого нуклеотида
    cover_dict['A', i+1] = 0
    cover_dict['C', i+1] = 0
    cover_dict['G', i+1] = 0
    cover_dict['T', i+1] = 0
    ref_cover_dict.append(cover_dict)
#ref_cover_dict


In [15]:
ref_cover_dict[199][('A', 200)] += 53 # не поняла как обновлять значения, теперь пишу условия когда это значение обновляется.
#ref_cover_dict
    

In [16]:
# попробую убрать гапы и soft clipping с помощью сигар.
seq  = 'CAGAAAACGACAAAGCTAGAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAGTATTACAGCCGCGCAAGG'
cigar  = '3M1D16M1D143M6S'
#print(len(seq), 'длина последовательности')

In [17]:
def working_cigar(cigar):
    '''For input the cigar string, for output the cigar dictionary'''
    cigar = list(cigar.strip()) #сделали список
    # для начала нужно сигар разбить на блоки и переделать цифры в инт. По идее олжно получиться int(61) str(S) int(37) str(M)...
    # Каждый символ пытаюсь перевести в инт, обработка исключений на буквы
    for i in range(len(cigar)): #пробуем каждый символ списка сделать интом
        try:
            cigar[i] = int(cigar[i])
        except ValueError:
            cigar[i] = cigar[i]
            
    # Потом объединяю цифры в одно число.
    cigar_new = []
    for i in range(len(cigar)): # если символ - строка - записываем его отдельно
        if type(cigar[i]) == str:
            cigar_new.append(cigar[i])                                    
        try:
            if type(cigar[i]) == type(cigar[i+1]) == type(cigar[i+2]) : # если три символа подряд это инт, объединяем их
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])+str(cigar[i+2])))
            if type(cigar[i]) == type(cigar[i+1]) and type(cigar[i]) != type(cigar[i+2]) and type(cigar[i-1]) ==str:  # если два символа подряд одного типа (такое может быть только с инт), объединяем их  
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])))
            if type(cigar[i+1]) == str and type(cigar[i-1]) == str: # если один инт - тоже записываем его
                cigar_new.append(cigar[i])
        except IndexError: # это нужно на случай окончания строки, list index out of range
            continue
    #print(cigar_new) # проверка правильности составления списка из cigar      
    # удобнее сделать словарь
    numbers = cigar_new[::2] # список с числом нуклеотидов
    clipping_type = [] # список с типом совпадений
    for i in range(1, len(cigar_new), 2): 
        clipping_type.append(cigar_new[i]);
    #print(numbers, clipping_type) # проверка правильности списков
    cigar_dict = [] # делаем словарь
    for i in range(len(numbers)):
        cigar_dict.append((numbers[i],clipping_type[i]))
    
    return cigar_dict
#working_cigar(cigar)

In [18]:
def seq_after_cigar(cigar, seq):
    nucl_number = 0 # сразу столбец из data
    print(seq)
    for e in cigar:
        #print(key, value) # првоерка работы функции working_cogar
        if e[1] == 'S':
            seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:]
        if e[1] == 'D':
            seq = seq[:nucl_number] + ' ' * e[0] + seq[nucl_number:]
        if e[1] == 'H':
            continue # это пока, может быть hard clipping не вырезается
        if e[1] == 'I':
            seq = seq[:nucl_number] + seq[nucl_number + e[0]:]
        nucl_number += e[0]
    #print(key, nucl_number)
    seq = seq.replace('N', '') 
    
    return seq
#print(seq_after_cigar(working_cigar(cigar), seq))

In [19]:
# попробуем теперь анализировать так все риды из таблицы.
new_data = []
for i in range(len(data())):
    new_data.append((data()['Начало рида'].iloc[i] - 1) * ' ' + seq_after_cigar(working_cigar(data()['CIGAR'].iloc[i]),data()['Рид'].iloc[i]))

...
CAAGGAGGAAATTTCGGATCGATGTAGTTAGAATCCAT
GTCTCTCGTGCGGCTGTAGTATTCAGGGCTGTGTTCTGGGATGTAGCTATTTTGTGAATATTCCTCGCAAGGAGGAAATTTCGGATCGATGTAGTTAGAATCCATCAAATACGAGCTCATGATCATTAATTTTTGGAGTGCAAAATATTTTTTCTACCTTTGTCGTTTTTCTGGTCCATAAAGAC
CTTTGTCGTTTTTCTGGTCCATAAAACCCTCCTACTAGCAAGCGACCCTGTAAAGTTACTTAACCACGTGACACAGCAGTCCAATGGCAGCTGGCGAGGTAGTTCCCGGATAAGGAACCAAATATTAATTCA
TCTGGGATGTAGCTATTTTGTGAATATTCCTCGCAAGGAGGAAATTTCGGATCGATGTAGTTAGAATCCATCAAATACGAGCTCATGATCATTAATTTTTGGAGTGAAAAATATTTTTTCTACCTTTGTCGTTTTTCTGGTCCATAAAGACCTCCTACTAGCAAGAGACCCTGTAAAGTTACT
GCTGTAATACTCGGGGCTGTGCTCGGGGATGTAGCTGTTTTGCGAATATTCCTCGCATGGAGGAAATTTCGGATCAATGTAGGTAGAGTCCATCAAATACGAGCTCATGATCATTAATTTCTGGAGTGGAAAATAATTTTTCTAGCTTTGT
GGTCACTCGCGGAGGATACAACTCCTGGTGGTGATGCTGGTAGCCAGTGTCTCTCGTGCGGCTGTAGTATTCAGGGCTGTGTTCTGGGATGTAGCTATTTTGTGAATATTCCTCGCAAGGAGGAAATTTCGGATCGATGTAGTTAGAATCCATCAAATACGAGCTCATGATCATTAATTTTTGGAGTGAAAAATATTTTTTCTACCTTTGTCGTTT
CTCTCGTGCGGCTGTAGTATTCAGGGCTGTGTTCTGGGATGTAGCTATTTTGTGAATATTCCTCGCAAGGAGGAAATTTCGGATC

In [21]:
ref_cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
for i in range(len(ref) + 1):
    cover_dict = {} # для каждого нуклеотида
    cover_dict['A, нуклеотид №', i+1] = 0
    cover_dict['C, нуклеотид №', i+1] = 0
    cover_dict['G, нуклеотид №', i+1] = 0
    cover_dict['T, нуклеотид №', i+1] = 0
    ref_cover_list.append(cover_dict)
for read in new_data:
    for nucl_number, nucl in enumerate(read):
            #print(nucl, nucl_number)
            if nucl == 'A':
                ref_cover_list[nucl_number][('A, нуклеотид №',nucl_number + 1)]  += 1
            if nucl == 'T':
                ref_cover_list[nucl_number][('T, нуклеотид №',nucl_number + 1)]  += 1
            if nucl == 'G':
                ref_cover_list[nucl_number][('G, нуклеотид №',nucl_number + 1)]  += 1
            if nucl == 'C':
                ref_cover_list[nucl_number][('C, нуклеотид №',nucl_number + 1)]  += 1
ref_cover_list

[{('A, нуклеотид №', 1): 0,
  ('C, нуклеотид №', 1): 0,
  ('G, нуклеотид №', 1): 0,
  ('T, нуклеотид №', 1): 305},
 {('A, нуклеотид №', 2): 0,
  ('C, нуклеотид №', 2): 0,
  ('G, нуклеотид №', 2): 305,
  ('T, нуклеотид №', 2): 0},
 {('A, нуклеотид №', 3): 0,
  ('C, нуклеотид №', 3): 0,
  ('G, нуклеотид №', 3): 0,
  ('T, нуклеотид №', 3): 305},
 {('A, нуклеотид №', 4): 0,
  ('C, нуклеотид №', 4): 0,
  ('G, нуклеотид №', 4): 309,
  ('T, нуклеотид №', 4): 0},
 {('A, нуклеотид №', 5): 0,
  ('C, нуклеотид №', 5): 3,
  ('G, нуклеотид №', 5): 0,
  ('T, нуклеотид №', 5): 309},
 {('A, нуклеотид №', 6): 0,
  ('C, нуклеотид №', 6): 0,
  ('G, нуклеотид №', 6): 0,
  ('T, нуклеотид №', 6): 319},
 {('A, нуклеотид №', 7): 0,
  ('C, нуклеотид №', 7): 331,
  ('G, нуклеотид №', 7): 0,
  ('T, нуклеотид №', 7): 0},
 {('A, нуклеотид №', 8): 0,
  ('C, нуклеотид №', 8): 0,
  ('G, нуклеотид №', 8): 18,
  ('T, нуклеотид №', 8): 321},
 {('A, нуклеотид №', 9): 0,
  ('C, нуклеотид №', 9): 0,
  ('G, нуклеотид №', 9)

Для одной конкретной сборки работает, попробую для другой.